In [15]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.metrics import confusion_matrix


In [3]:
df = pd.read_csv("../../trainLabels.csv")

In [4]:
n = len(df)
samples = {}
X_train = np.nan
y_train = []
i = 0
while i < 1000:
    index = np.random.randint(0, n)
    tmp = plt.imread("../../train256/"+df.iloc[index]['image']+".jpeg")
    if (tmp.shape == (171, 256, 3)) and (df.iloc[index]['image'] not in samples):
        samples[df.iloc[index]['image']] = df.iloc[index]['level']
        #print plt.imread("../../train256/"+df.iloc[index]['image']+".jpeg").shape
        tmp = tmp.reshape((1, 171*256*3))
        if X_train is np.nan:
            X_train = tmp
        else:
            X_train = np.concatenate((X_train, tmp), axis=0)
        y_train.append(df.iloc[index]['level'])
        i += 1
y_train = np.array(y_train)

In [5]:
n = len(df)
tests = {}
X_test = np.nan
y_test = []
i = 0
while i < 1000:
    index = np.random.randint(0, n)
    tmp = plt.imread("../../train256/"+df.iloc[index]['image']+".jpeg")
    if (tmp.shape == (171, 256, 3)) and (df.iloc[index]['image'] not in samples) and (df.iloc[index]['image'] not in tests):
        tests[df.iloc[index]['image']] = df.iloc[index]['level']
        tmp = tmp.reshape((1, 171*256*3))
        if X_test is np.nan:
            X_test = tmp
        else:
            X_test = np.concatenate((X_test, tmp), axis=0)
        y_test.append(df.iloc[index]['level'])
        i += 1
y_test = np.array(y_test)

In [6]:
rf = ensemble.RandomForestClassifier(n_estimators = 500, n_jobs = -1)


In [7]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0)

In [14]:
y_pred = rf.predict(X_test)

In [8]:
rf.score(X_test, y_test)

0.72199999999999998

In [19]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(y_test))
    plt.xticks(tick_marks, "", rotation=45)
    plt.yticks(tick_marks, "")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [25]:
# Compute confusion matrix
confusion_matrix(y_test, y_pred)
#np.set_printoptions(precision=2)
#print('Confusion matrix, without normalization')
print(cm)
#plt.figure()
#plot_confusion_matrix(cm)

[[718   0  10   0   0]
 [ 56   0   2   0   0]
 [172   0   3   0   0]
 [ 18   0   0   1   0]
 [ 19   0   1   0   0]]


In [144]:
X_train.shape

(1000, 131328)

In [145]:
y_train.shape

(1000,)

In [149]:
X_test.shape

(1000, 131328)

In [150]:
y_test.shape

(1000,)

In [153]:
y_test

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 2, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 1, 2, 0, 2, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       2, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 1, 4, 0,
       0, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 3, 0, 4, 0, 0, 1, 2, 0, 0, 4, 0,
       4, 0, 0, 0, 0, 0, 3, 0, 1, 0, 2, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 4, 0, 0, 0, 0, 0, 1, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 4,
       0, 2, 3, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 0, 4,
       0, 0,

In [12]:
plt.hist(y_train)
plt.show()

In [13]:
plt.hist(y_test)
plt.show()